## <font color="FF8C00"> **QUESTÃO DE PESQUISA**: </font>

#### <font color="FFDEAD"> QUAIS SÃO OS FATORES QUE ESTÃO MAIS ASSOCIADOS AO DESEMPENHO ESCOLAR EM MATEMÁTICA DOS ALUNOS DO 9º ANO DO ENISNO FUNDAMENTAL, DE ESCOLAS BRASILEIRAS, BASEADO NOS DADOS DO SAEB 2019?</font>

In [73]:
# A melhor forma de encontrar os fatores
# Uma abordagem ou uma metodologia para abordar os fatores
# Teste de associação (qui-quadrado)
# Comece com a correlação: pearson, spearman, kendall e crammer (categorico)... 

In [74]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Alunos

In [75]:
#alunos = pd.read_csv("TS_ALUNO_9EF.csv")
#alunos.shape

In [76]:
#alunos.head()

## Professores

In [77]:
#profe = pd.read_csv("TS_PROFESSOR.csv")
#profe.shape

In [78]:
#profe.head()

## Diretores

In [79]:
# ID_REGIAO (NUM) Código da Região [1- Norte, 2 - Nordeste, 3 - Sudeste, 4 - Sul, 5 - Centro-Oeste]
# ID_UF (NUM) [11-RO, 12-AC, 13-AM, 14-RR, 15-PA, 16-AP, 17-TO, 21-MA, 22-PI, 23-CE, 24-RN, 25-PB, 26-PE, 27-AL, 28-SE, 29-BA, 31-MG, 32-ES, 33-RJ, 35-SP, 41-PR, 42-SC, 43-RS, 50-MS, 51-MT, 52-GO, 53-DF]
# ID_MUNICIPIO (NUM) Código do Município
# ID_AREA (NUM) [1 - Capital, 2 - Interior] Área
# ID_ESCOLA (NUM) Código da Escola OK
# ID_DEPENDENCIA_ADM (NUM) Dependência Administrativa [1 - Federal, 2 - Estadual, 3 - Municipal, 4 - Privada]
# ID_LOCALIZACAO (NUM) Localização [1 - Urbana, 2 - Rural]
# IN_PREENCHIMENTO_QUESTIONARIO (NUM) Indicador de preenchimento do questionário [0 - Não Preenchido, 1 - Preenchido Parcial ou Totalmente] OK

# 1 - Informações dos Diretores
# TX_RESP_Q001 (CHAR) Qual é a sua cor ou raça? [a,b,c,d,e,f]
# TX_RESP_Q002 (NUM) Por quanto tempo você trabalhou como professor(a) antes de se tornar diretor(a)? Anos (Regra de Validação: Q002 >= 0 e Q002 <= 35)
# TX_RESP_Q004 (NUM) Você possui quanto tempo de experiência como diretor(a) de escola? Anos (Regra de Validação: Q004 >= 0 e Q004 <= 35)
# TX_RESP_Q009 (NUM) Quantas horas você trabalha em uma semana normal em atividades relacionadas à educação?(Regra de Validação: Q009 >= 0 e Q009 <= 70)
# TX_RESP_Q012 (NUM) Em uma semana normal de trabalho, quantas horas você costuma gastar, aproximadamente, com as seguintes atividades para esta escola: Reunião com professores(as) (Regra de Validação: Q012 >= 0 e Q012 <= 20)
# TX_RESP_Q013 (NUM) Em uma semana normal de trabalho, quantas horas você costuma gastar, aproximadamente, com as seguintes atividades para esta escola: Atendimento aos pais ou responsáveis (Regra de Validação: Q013 >= 0 e Q013 <= 30) OK
# TX_RESP_Q015 (NUM) Em uma semana normal de trabalho, quantas horas você costuma gastar, aproximadamente, com as seguintes atividades para esta escola: Atendimento aos(às) alunos(as) (Regra de Validação: Q015 >= 0 e Q015 <= 30)

# 2 - Condições de Funcionamento da Escola
# TX_RESP_Q041 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Os recursos financeiros foram suficientes. [a,b,c,d] OK
# TX_RESP_Q042 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Havia professores(as) para todas as disciplinas. [a,b,c,d] OK
# TX_RESP_Q043 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Havia quantidade suficiente de pessoal administrativo. [a,b,c,d] OK
# TX_RESP_Q044 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Havia quantidade suficiente de pessoal para apoio pedagógico (coordenador, orientador etc.). [a,b,c,d] OK
# TX_RESP_Q045 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Os recursos pedagógicos foram suficientes. [a,b,c,d] OK
# TX_RESP_Q047 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Os(As) professores(as) foram assíduos(as). [a,b,c,d] OK
# TX_RESP_Q050 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: Os(As) alunos(as) foram assíduos(as). [a,b,c,d] OK

# TX_RESP_Q052 (CHAR) Indique o quanto concorda ou discorda das afirmativas relativas às condições de funcionamento desta escola neste ano: A comunidade apoiou a gestão da escola. [a,b,c,d]

#TX_RESP_Q057 (CHAR) O calendário escolar pré-estabelecido foi cumprido sem interrupções? [a,b] OK


# 4 - Gestão e Participação
# TX_RESP_Q118 (CHAR) O Conselho Escolar é um colegiado geralmente constituído por representantes da escola e da comunidade que tem como objetivo acompanhar as atividades escolares. Na sua escola existe Conselho Escolar? [a,b,c]
# TX_RESP_Q119 (NUM) Quantas reuniões do Conselho Escolar ocorreram neste ano? (Regra de Dependência: somente será preenchida caso Q118 seja igual a "A")
# TX_RESP_Q120 (NUM) Considere os atores relacionados a seguir e indique quantos participam do Conselho Escolar: Professores(as) (Regra de Dependência: somente será preenchida caso Q118 seja igual a "A")
# TX_RESP_Q121 (NUM) Considere os atores relacionados a seguir e indique quantos participam do Conselho Escolar: Alunos(as) (Regra de Dependência: somente será preenchida caso Q118 seja igual a "A")
# TX_RESP_Q130 (NUM) O Conselho de Classe é um órgão formado por todos os professores que lecionam em cada turma/ano. Neste ano e nesta escola, quantas vezes se reuniu o Conselho de Classe? OK
# TX_RESP_Q134 (CHAR) Há Grêmio Estudantil? (Regra de Dependência: somente será preenchida caso Q037 ou Q038 seja igual a  "A") [a,b,c,d]
# TX_RESP_Q137 (CHAR) A escola segue orientação religiosa (Regra de Dependência: somente será preenchida caso Q035, Q036, Q07 ou Q038 seja igual a "A"). [a,b]

# TX_RESP_Q146 (CHAR) A escola oferece merenda aos(às) estudantes? [a,b]
# TX_RESP_Q150 (CHAR) A quantidade de alimentos é suficiente para todos(as) (Regra de Dependência: somente será preenchida caso Q146 seja igual a "A). [a,b,c,d]
# TX_RESP_Q151 (CHAR) Os alimentos são de boa qualidade (Regra de Dependência: somente será preenchida caso Q146 seja igual a "A).

# 5 - Gestão Pedagógica
# TX_RESP_Q158 (CHAR) A escola possui Projeto Político-Pedagógico? [a,b]
# TX_RESP_Q163 (CHAR) Estabelece metas de aprendizagem? (Regra de Dependência: somente será preenchida caso Q158 seja igual a "A") [a,b]

# TX_RESP_Q224 (CHAR) Indique se neste ano a escola ofereceu atividades de formação nas seguintes áreas: Avaliação da aprendizagem. [a,b]
# TX_RESP_Q225 (CHAR) Indique se neste ano a escola ofereceu atividades de formação nas seguintes áreas: Avaliação em larga escala. [a,b]

In [80]:
dir = pd.read_csv("TS_DIRETOR.csv")
dir = dir.loc[:, ["ID_ESCOLA", "IN_PREENCHIMENTO_QUESTIONARIO", "TX_RESP_Q002", "TX_RESP_Q004", "TX_RESP_Q009", "TX_RESP_Q012", 
                "TX_RESP_Q013", "TX_RESP_Q015", "TX_RESP_Q041", "TX_RESP_Q042", "TX_RESP_Q043", "TX_RESP_Q044", "TX_RESP_Q045", "TX_RESP_Q047",
                "TX_RESP_Q050", "TX_RESP_Q052", "TX_RESP_Q057", "TX_RESP_Q118", "TX_RESP_Q120", "TX_RESP_Q121", "TX_RESP_Q130", "TX_RESP_Q137", 
                "TX_RESP_Q146", "TX_RESP_Q150", "TX_RESP_Q151", "TX_RESP_Q158", "TX_RESP_Q163", "TX_RESP_Q224", "TX_RESP_Q225"]]

dir = dir[dir["IN_PREENCHIMENTO_QUESTIONARIO"] == 1]
dir.drop(["IN_PREENCHIMENTO_QUESTIONARIO"], axis=1, inplace=True)

# 1 - Informações dos Diretores
# ID_ESCOLA
# COR [A,B,C,D,E,F] => ["Branca", "Preta", "Parda", "Amarela", "Indígena", "Não quero declarar"]
# ANOS_EXPERIENCIA_PROFESSOR [Q002 >= 0 e Q002 <= 35]
# ANOS_EXPERIENCIA_DIRETOR [Q004 >= 0 e Q004 <= 35]
# HORAS_TRABALHOSEMANAL_EDUCACAO [Q009 >= 0 e Q009 <= 70]
# HORAS_REUNIAOPROFESSORES [Q012 >= 0 e Q012 <= 20]
# HORAS_ATENDIMENTO_AOSPAIS [Q013 >= 0 e Q013 <= 30]
# HORAS_ATENDIMENTO_ALUNOS [Q015 >= 0 e Q015 <= 30]


# 2 - Condições de Funcionamento da Escola
# RECURSOSFINANCEIROS_SUFICIENTES [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# PROFESSORES_TODASDISCIPLINAS [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# PESSOALADMINISTRATIVO_SUFICIENTES [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# APOIOPEDAGOGICO_SUFICIENTES [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# RECURSOSPEDAGOGICOS_SUFICIENTES [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# PROFESSORES_ASSIDUOS [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# ALUNOS_ASSIDUOS [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# APOIOCOMUNIDADE_GESTAO [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# INTERRUPCOES_CALENDARIO [A,B] => ["Sim", "Não"]

# 4 - Gestão e Participação
# CONSELHO_ESCOLAR [A,B,C] => ["Sim, está ativo", "Sim, está inativo", "Não existe"]

# QUANT_PROFESSORES_PARTICIPA_CS
# QUANT_ALUNOS_PARTICIPA_CS 
# QUANT_REUNIOES_CONSELHOCLASSE
# ESCOLA_ORIENTACAO_RELIGIOSA [A, B] => ["Sim", "Não"]

# ESCOLA_MERENDA [A, B] => ["Sim", "Não"] 
# MERENDA_SUFICIENTE [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 
# MERENDA_QUALIDADE [A,B,C,D] => ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"] 

# 5 - Gestão Pedagógica
# PROJETO_POLITICO_PEDAGOGICO [A,B] => ["Sim", "Não"]
# METAS_APRENDIZAGEM [A,B] => ["Sim", "Não"]
# ATIVIDADES_AVALIACAO_APRENDIZAGEM [A,B] => ["Sim", "Não"]
# ATIVIDADES_AVALIACAO_LARGAESCALA [A,B] => ["Sim", "Não"]

#dir["TX_RESP_Q001"] = dir["TX_RESP_Q001"].replace(["A", "B", "C", "D", "E", "F"], ["Branca", "Preta", "Parda", "Amarela", "Indígena", "Não quero declarar"])
dir["TX_RESP_Q041"] = dir["TX_RESP_Q041"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q042"] = dir["TX_RESP_Q042"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q043"] = dir["TX_RESP_Q043"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q044"] = dir["TX_RESP_Q044"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q045"] = dir["TX_RESP_Q045"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q047"] = dir["TX_RESP_Q047"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q050"] = dir["TX_RESP_Q050"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q052"] = dir["TX_RESP_Q052"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q057"] = dir["TX_RESP_Q057"].replace(["A", "B"], ["Sim", "Não"])
dir["TX_RESP_Q118"] = dir["TX_RESP_Q118"].replace(["A", "B", "C"], ["Sim, existe e está ativo", "Sim, existe, mas está inátivo", "Não existe"])

dir["TX_RESP_Q137"] = dir["TX_RESP_Q137"].replace(["A", "B"], ["Sim", "Não"])
dir["TX_RESP_Q146"] = dir["TX_RESP_Q146"].replace(["A", "B"], ["Sim", "Não"])
dir["TX_RESP_Q150"] = dir["TX_RESP_Q150"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q151"] = dir["TX_RESP_Q151"].replace(["A", "B", "C", "D"], ["Discordo Fortemente", "Discordo", "Concordo", "Concordo Fortemente"])
dir["TX_RESP_Q158"] = dir["TX_RESP_Q158"].replace(["A", "B"], ["Sim", "Não"])
dir["TX_RESP_Q163"] = dir["TX_RESP_Q163"].replace(["A", "B", "C"], ["Sim", "Não", "Não se aplica"])
dir["TX_RESP_Q224"] = dir["TX_RESP_Q224"].replace(["A", "B"], ["Sim", "Não"]) 
dir["TX_RESP_Q225"] = dir["TX_RESP_Q225"].replace(["A", "B"], ["Sim", "Não"])


#dir.columns = ["ID_ESCOLA",
#               "ANOS_EXPERIENCIA_PROFESSOR",
#              "ANOS_EXPERIENCIA_DIRETOR",
#               "HORAS_TRABALHOSEMANAL_EDUCACAO",
#               "HORAS_REUNIAOPROFESSORES",
#               "HORAS_ATENDIMENTO_AOSPAIS",
#               "HORAS_ATENDIMENTO_ALUNOS", 
#               "RECURSOSFINANCEIROS_SUFICIENTES", 
#               "PROFESSORES_TODASDISCIPLINAS", 
#               "PESSOALADMINISTRATIVO_SUFICIENTES", 
#               "APOIOPEDAGOGICO_SUFICIENTES", 
#               "RECURSOSPEDAGOGICOS_SUFICIENTES", 
#               "PROFESSORES_ASSIDUOS", 
#               "ALUNOS_ASSIDUOS",
#               "APOIOCOMUNIDADE_GESTAO",
#               "INTERRUPCOES_CALENDARIO",
#               "CONSELHO_ESCOLAR",
#               "QUANT_PROFESSORES_PARTICIPA_CS", 
#               "QUANT_ALUNOS_PARTICIPA_CS", 
#               "QUANT_REUNIOES_CONSELHOCLASSE", 
#               "ESCOLA_ORIENTACAO_RELIGIOSA",
#               "ESCOLA_MERENDA", 
#               "MERENDA_SUFICIENTE", 
#               "MERENDA_QUALIDADE", 
#               "PROJETO_POLITICO_PEDAGOGICO", 
#               "METAS_APRENDIZAGEM", 
#               "ATIVIDADES_AVALIACAO_APRENDIZAGEM", 
#               "ATIVIDADES_AVALIACAO_LARGAESCALA"]

dir.shape

(67886, 28)

In [81]:
dir.head()

,ID_ESCOLA,TX_RESP_Q002,TX_RESP_Q004,TX_RESP_Q009,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q015,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,...,TX_RESP_Q121,TX_RESP_Q130,TX_RESP_Q137,TX_RESP_Q146,TX_RESP_Q150,TX_RESP_Q151,TX_RESP_Q158,TX_RESP_Q163,TX_RESP_Q224,TX_RESP_Q225
0,11024666,18.0,0.0,40.0,2.0,2.0,NaN,Discordo Fortemente,Discordo,Discordo,...,3.0,3.0,Não,Sim,Concordo,Concordo,Sim,Sim,Sim,Não
1,11024682,14.0,11.0,50.0,2.0,10.0,8.0,Concordo,Discordo Fortemente,Discordo,...,NaN,3.0,Não,Sim,Concordo,Concordo,Sim,Sim,Sim,Sim
2,11024828,5.0,4.0,40.0,2.0,2.0,4.0,Concordo,Concordo,Discordo,...,NaN,2.0,Não,Sim,Concordo Fortemente,Concordo Fortemente,Sim,Sim,Sim,Sim
3,11024968,5.0,0.0,40.0,2.0,4.0,10.0,Concordo,Concordo,Concordo,...,2.0,3.0,Não,Sim,Concordo,Concordo,Sim,Sim,Sim,Sim
4,11025077,9.0,NaN,40.0,2.0,4.0,6.0,Discordo,Concordo,Concordo,...,NaN,3.0,Não,Sim,Discordo,Concordo,Sim,Sim,Não,Não


## Escolas

In [82]:
# ID_REGIAO (NUM) Código da Região [1- Norte, 2 - Nordeste, 3 - Sudeste, 4 - Sul, 5 - Centro-Oeste]
# ID_UF (NUM) [11-RO, 12-AC, 13-AM, 14-RR, 15-PA, 16-AP, 17-TO, 21-MA, 22-PI, 23-CE, 24-RN, 25-PB, 26-PE, 27-AL, 28-SE, 29-BA, 31-MG, 32-ES, 33-RJ, 35-SP, 41-PR, 42-SC, 43-RS, 50-MS, 51-MT, 52-GO, 53-DF]
# ID_MUNICIPIO (NUM) Código do Município
# ID_AREA (NUM) [1 - Capital, 2 - Interior] Área
# ID_ESCOLA (NUM) Código da Escola OK


# ID_DEPENDENCIA_ADM (NUM) Dependência Administrativa [1 - Federal, 2 - Estadual, 3 - Municipal, 4 - Privada] OK
# ID_LOCALIZACAO (NUM) Localização [1 - Urbana, 2 - Rural] OK
# PC_FORMACAO_DOCENTE_FINAL (NUM) Indicador de Adequação da Formação Docente (Informação referente ao Grupo 1, para os Anos Finais do EF)
# NIVEL_SOCIO_ECONOMICO (CHAR) Indicador de Nível Socioeconômico (Inse) [Nivel I, Nivel II, Nivel III, Nivel VI, Nivel V, Nivel VI, Nivel VII]

# Nivel_0_LP9 (NUM) (0, 200)
# Nivel_1_LP9 (NUM) (200, 225)
# Nivel_2_LP9 (NUM) (225, 250)
# Nivel_3_LP9 (NUM) (250, 275)
# Nivel_4_LP9 (NUM) (275, 300)
# Nivel_5_LP9 (NUM) (300, 325)
# Nivel_6_LP9 (NUM) (325, 350)
# Nivel_7_LP9 (NUM) (350, 375)
# Nivel_8_LP9 (NUM) (375, +)
# Nivel_0_MT9 (NUM) (0, 200)
# Nivel_1_MT9 (NUM) (200, 225)
# Nivel_2_MT9 (NUM) (225, 250)
# Nivel_3_MT9 (NUM) (250, 275)
# Nivel_4_MT9 (NUM) (275, 300)
# Nivel_5_MT9 (NUM) (300, 325)
# Nivel_6_MT9 (NUM) (325, 350)
# Nivel_7_MT9 (NUM) (350, 375)
# Nivel_8_MT9 (NUM) (375, 400)
# Nivel_9_MT9 (NUM) (400, +)

# MEDIA_9EF_LP (NUM) Média em Língua Portuguesa 9º ano
# MEDIA_9EF_MT (NUM) Média em Matemática 9º ano
 

In [83]:
esc = pd.read_csv("TS_ESCOLA.csv", encoding = "ISO-8859-1", engine='python')
esc = esc.loc[:, ["ID_ESCOLA", "ID_LOCALIZACAO", "PC_FORMACAO_DOCENTE_FINAL", "NIVEL_SOCIO_ECONOMICO", "MEDIA_9EF_MT", "MEDIA_9EF_LP"]]
esc.shape

(70606, 6)

In [84]:
esc.head()

,ID_ESCOLA,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_FINAL,NIVEL_SOCIO_ECONOMICO,MEDIA_9EF_MT,MEDIA_9EF_LP
0,11024666,2,64.4,Nível IV,NaN,NaN
1,11024682,1,80.4,Nível IV,NaN,NaN
2,11024828,1,48.9,Nível IV,244.94,239.76
3,11024968,1,NaN,Nível IV,NaN,NaN
4,11025077,1,29.5,Nível IV,NaN,NaN


## Juntando Dataset de Diretores e Escolas

In [85]:
diresc = pd.merge(dir, esc, how = 'inner', on = 'ID_ESCOLA')
diresc.shape

(66017, 33)

In [86]:
diresc = diresc.replace("*", np.nan)
diresc = diresc.replace(".", np.nan)
diresc = diresc.replace(" ", np.nan)

In [87]:
diresc[diresc.columns[diresc.isnull().any()]].isnull().sum()

TX_RESP_Q002                  2398
TX_RESP_Q004                  5662
TX_RESP_Q009                  3856
TX_RESP_Q012                  2446
TX_RESP_Q013                  3002
TX_RESP_Q015                  4275
TX_RESP_Q041                   142
TX_RESP_Q042                   180
TX_RESP_Q043                   201
TX_RESP_Q044                   158
TX_RESP_Q045                   223
TX_RESP_Q047                   251
TX_RESP_Q050                   230
TX_RESP_Q052                   285
TX_RESP_Q057                   319
TX_RESP_Q118                   292
TX_RESP_Q120                 10789
TX_RESP_Q121                 24352
TX_RESP_Q130                 10732
TX_RESP_Q137                   472
TX_RESP_Q146                   331
TX_RESP_Q150                   889
TX_RESP_Q151                   947
TX_RESP_Q158                   304
TX_RESP_Q163                  3811
TX_RESP_Q224                   517
TX_RESP_Q225                  1937
PC_FORMACAO_DOCENTE_FINAL    26159
NIVEL_SOCIO_ECONOMIC

In [88]:
diresc.dropna(axis=0, inplace=True)
diresc.drop_duplicates(keep='last', inplace=True)
diresc.shape

(14842, 33)

In [89]:
diresc.head()

,ID_ESCOLA,TX_RESP_Q002,TX_RESP_Q004,TX_RESP_Q009,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q015,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,...,TX_RESP_Q151,TX_RESP_Q158,TX_RESP_Q163,TX_RESP_Q224,TX_RESP_Q225,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_FINAL,NIVEL_SOCIO_ECONOMICO,MEDIA_9EF_MT,MEDIA_9EF_LP
5,11025280,20.0,4.0,40.0,1.0,3.0,6.0,Concordo,Discordo,Concordo,...,Concordo Fortemente,Sim,Sim,Não,Não,1,36.5,Nível IV,241.78,240.09
6,11025310,7.0,9.0,40.0,1.0,1.0,20.0,Discordo,Concordo,Concordo,...,Concordo Fortemente,Sim,Sim,Sim,Sim,1,55.6,Nível III,265.14,276.42
9,11025638,23.0,7.0,40.0,1.0,2.0,2.0,Concordo,Discordo Fortemente,Discordo Fortemente,...,Concordo,Sim,Sim,Sim,Sim,1,72.9,Nível IV,273.59,268.37
10,11006773,12.0,16.0,40.0,4.0,4.0,3.0,Concordo,Discordo Fortemente,Discordo Fortemente,...,Concordo,Sim,Sim,Sim,Sim,1,60.9,Nível IV,257.19,251.32
12,11006854,6.0,5.0,10.0,2.0,3.0,3.0,Discordo,Discordo,Concordo,...,Concordo,Sim,Sim,Sim,Não,1,46.9,Nível IV,245.92,244.61


In [90]:
escala = [
    (diresc["MEDIA_9EF_MT"] < 200),
    (diresc["MEDIA_9EF_MT"] >= 200) & (diresc["MEDIA_9EF_MT"] < 225),
    (diresc["MEDIA_9EF_MT"] >= 225) & (diresc["MEDIA_9EF_MT"] < 250),
    (diresc["MEDIA_9EF_MT"] >= 250) & (diresc["MEDIA_9EF_MT"] < 275),
    (diresc["MEDIA_9EF_MT"] >= 275) & (diresc["MEDIA_9EF_MT"] < 300),
    (diresc["MEDIA_9EF_MT"] >= 300) & (diresc["MEDIA_9EF_MT"] < 325),
    (diresc["MEDIA_9EF_MT"] >= 325) & (diresc["MEDIA_9EF_MT"] < 350),
    (diresc["MEDIA_9EF_MT"] >= 350) & (diresc["MEDIA_9EF_MT"] < 375),
    (diresc["MEDIA_9EF_MT"] >= 375) & (diresc["MEDIA_9EF_MT"] < 400),
    (diresc["MEDIA_9EF_MT"] >= 400)
    ]
nivel = ["Nivel 0", "Nivel 1", "Nivel 2", "Nivel 3", "Nivel 4", "Nivel 5", "Nivel 6", "Nivel 7", "Nivel 8", "Nivel 9"]
diresc["ESCALA_MT"] = np.select(escala, nivel)

In [91]:
escala = [
    (diresc["MEDIA_9EF_LP"] < 200),
    (diresc["MEDIA_9EF_LP"] >= 200) & (diresc["MEDIA_9EF_LP"] < 225),
    (diresc["MEDIA_9EF_LP"] >= 225) & (diresc["MEDIA_9EF_LP"] < 250),
    (diresc["MEDIA_9EF_LP"] >= 250) & (diresc["MEDIA_9EF_LP"] < 275),
    (diresc["MEDIA_9EF_LP"] >= 275) & (diresc["MEDIA_9EF_LP"] < 300),
    (diresc["MEDIA_9EF_LP"] >= 300) & (diresc["MEDIA_9EF_LP"] < 325),
    (diresc["MEDIA_9EF_LP"] >= 325) & (diresc["MEDIA_9EF_LP"] < 350),
    (diresc["MEDIA_9EF_LP"] >= 350) & (diresc["MEDIA_9EF_LP"] < 375),
    (diresc["MEDIA_9EF_LP"] >= 375)
    ]
nivel = ["Nivel 0", "Nivel 1", "Nivel 2", "Nivel 3", "Nivel 4", "Nivel 5", "Nivel 6", "Nivel 7", "Nivel 8"]
diresc["ESCALA_LP"] = np.select(escala, nivel)

In [92]:
diresc["ESCALA_LP"].value_counts()

Nivel 3    7264
Nivel 2    4417
Nivel 4    2206
Nivel 1     743
Nivel 5     154
Nivel 0      52
Nivel 6       6
Name: ESCALA_LP, dtype: int64

In [93]:
diresc["ESCALA_MT"].value_counts()

Nivel 3    6807
Nivel 2    4151
Nivel 4    2669
Nivel 1     698
Nivel 5     412
Nivel 6      50
Nivel 0      45
Nivel 7      10
Name: ESCALA_MT, dtype: int64

In [94]:
diresc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14842 entries, 5 to 65926
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID_ESCOLA                  14842 non-null  int64  
 1   TX_RESP_Q002               14842 non-null  float64
 2   TX_RESP_Q004               14842 non-null  float64
 3   TX_RESP_Q009               14842 non-null  float64
 4   TX_RESP_Q012               14842 non-null  float64
 5   TX_RESP_Q013               14842 non-null  float64
 6   TX_RESP_Q015               14842 non-null  float64
 7   TX_RESP_Q041               14842 non-null  object 
 8   TX_RESP_Q042               14842 non-null  object 
 9   TX_RESP_Q043               14842 non-null  object 
 10  TX_RESP_Q044               14842 non-null  object 
 11  TX_RESP_Q045               14842 non-null  object 
 12  TX_RESP_Q047               14842 non-null  object 
 13  TX_RESP_Q050               14842 non-null  obj

In [95]:
catg = diresc.select_dtypes(include="object").columns.tolist()

for index in catg:
    diresc[index] = diresc[index].astype("category")

In [96]:
diresc = diresc.iloc[:,:].reset_index(drop=True)
diresc.shape

(14842, 35)

In [97]:
diresc.head()

,ID_ESCOLA,TX_RESP_Q002,TX_RESP_Q004,TX_RESP_Q009,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q015,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,...,TX_RESP_Q163,TX_RESP_Q224,TX_RESP_Q225,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_FINAL,NIVEL_SOCIO_ECONOMICO,MEDIA_9EF_MT,MEDIA_9EF_LP,ESCALA_MT,ESCALA_LP
0,11025280,20.0,4.0,40.0,1.0,3.0,6.0,Concordo,Discordo,Concordo,...,Sim,Não,Não,1,36.5,Nível IV,241.78,240.09,Nivel 2,Nivel 2
1,11025310,7.0,9.0,40.0,1.0,1.0,20.0,Discordo,Concordo,Concordo,...,Sim,Sim,Sim,1,55.6,Nível III,265.14,276.42,Nivel 3,Nivel 4
2,11025638,23.0,7.0,40.0,1.0,2.0,2.0,Concordo,Discordo Fortemente,Discordo Fortemente,...,Sim,Sim,Sim,1,72.9,Nível IV,273.59,268.37,Nivel 3,Nivel 3
3,11006773,12.0,16.0,40.0,4.0,4.0,3.0,Concordo,Discordo Fortemente,Discordo Fortemente,...,Sim,Sim,Sim,1,60.9,Nível IV,257.19,251.32,Nivel 3,Nivel 3
4,11006854,6.0,5.0,10.0,2.0,3.0,3.0,Discordo,Discordo,Concordo,...,Sim,Sim,Não,1,46.9,Nível IV,245.92,244.61,Nivel 2,Nivel 2


## MATEMÁTICA

In [98]:
mat = diresc.drop(columns=["ID_ESCOLA", "MEDIA_9EF_LP", "ESCALA_LP"], axis=1) 
#mat.drop(mat[mat["MEDIA_9EF_MT"] < 200].index, axis=0, inplace=True)
#mat.drop(columns=["MEDIA_9EF_MT"], axis=1, inplace=True) 
mat.shape

(14842, 32)

In [99]:
mat.head()

,TX_RESP_Q002,TX_RESP_Q004,TX_RESP_Q009,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q015,TX_RESP_Q041,TX_RESP_Q042,TX_RESP_Q043,TX_RESP_Q044,...,TX_RESP_Q151,TX_RESP_Q158,TX_RESP_Q163,TX_RESP_Q224,TX_RESP_Q225,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_FINAL,NIVEL_SOCIO_ECONOMICO,MEDIA_9EF_MT,ESCALA_MT
0,20.0,4.0,40.0,1.0,3.0,6.0,Concordo,Discordo,Concordo,Concordo,...,Concordo Fortemente,Sim,Sim,Não,Não,1,36.5,Nível IV,241.78,Nivel 2
1,7.0,9.0,40.0,1.0,1.0,20.0,Discordo,Concordo,Concordo,Concordo,...,Concordo Fortemente,Sim,Sim,Sim,Sim,1,55.6,Nível III,265.14,Nivel 3
2,23.0,7.0,40.0,1.0,2.0,2.0,Concordo,Discordo Fortemente,Discordo Fortemente,Discordo Fortemente,...,Concordo,Sim,Sim,Sim,Sim,1,72.9,Nível IV,273.59,Nivel 3
3,12.0,16.0,40.0,4.0,4.0,3.0,Concordo,Discordo Fortemente,Discordo Fortemente,Discordo Fortemente,...,Concordo,Sim,Sim,Sim,Sim,1,60.9,Nível IV,257.19,Nivel 3
4,6.0,5.0,10.0,2.0,3.0,3.0,Discordo,Discordo,Concordo,Concordo,...,Concordo,Sim,Sim,Sim,Não,1,46.9,Nível IV,245.92,Nivel 2


In [100]:
mat.describe()

,TX_RESP_Q002,TX_RESP_Q004,TX_RESP_Q009,TX_RESP_Q012,TX_RESP_Q013,TX_RESP_Q015,TX_RESP_Q120,TX_RESP_Q121,TX_RESP_Q130,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_FINAL,MEDIA_9EF_MT
count,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000,14842.000000
mean,14.071149,6.628217,44.161299,3.629834,5.547029,6.687576,5.072766,3.545210,3.254683,1.156852,63.715153,259.163897
std,7.164422,6.113312,11.633743,2.412966,3.885322,4.923280,2.845355,2.224302,1.738347,0.363674,20.778300,21.318618
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,175.680000
25%,9.000000,2.000000,40.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,1.000000,52.500000,245.420000
50%,14.000000,5.000000,40.000000,3.000000,5.000000,5.000000,4.000000,3.000000,3.000000,1.000000,68.500000,258.820000
75%,19.000000,10.000000,50.000000,4.000000,7.000000,8.000000,8.000000,5.000000,4.000000,1.000000,78.300000,272.427500
max,35.000000,35.000000,70.000000,20.000000,30.000000,30.000000,10.000000,10.000000,10.000000,2.000000,100.000000,372.100000


## TRANSFORMAÇÕES

In [101]:
catOrdinais = ['TX_RESP_Q041', 'TX_RESP_Q042', 'TX_RESP_Q043', 'TX_RESP_Q044', 'TX_RESP_Q045', 'TX_RESP_Q047', 'TX_RESP_Q050', 'TX_RESP_Q052', 'TX_RESP_Q150', 'TX_RESP_Q151', 'NIVEL_SOCIO_ECONOMICO']

In [102]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
mat[catOrdinais] = enc.fit_transform(mat[catOrdinais])

## CORRELAÇÕES

### Pearson

In [103]:
# Verificando a correlação entre as váriaveis 

#plt.figure(figsize=(12,8))
#sns.heatmap(mat_encode.corr(method='pearson'), annot=True, cmap="PiYG")
#plt.title("Correlação entre Variáveis")
#plt.show()

### Spearman

In [104]:
#plt.figure(figsize=(12,8))
#sns.heatmap(mat_encode.corr(method='spearman'), annot=True, cmap="PiYG")
#plt.title("Correlação entre Variáveis")
#plt.show()

In [105]:
from scipy.stats.stats import spearmanr

#cat = ['TX_RESP_Q002', 'TX_RESP_Q004', 'TX_RESP_Q009', 'TX_RESP_Q012',
#       'TX_RESP_Q013', 'TX_RESP_Q015', 'TX_RESP_Q041', 'TX_RESP_Q042',
#       'TX_RESP_Q043', 'TX_RESP_Q044', 'TX_RESP_Q045', 'TX_RESP_Q047',
#       'TX_RESP_Q050', 'TX_RESP_Q052', 'TX_RESP_Q057', 'TX_RESP_Q120', 'TX_RESP_Q121', 'TX_RESP_Q130', 'TX_RESP_Q137',
#       'TX_RESP_Q150', 'TX_RESP_Q151', 'TX_RESP_Q163', 'TX_RESP_Q224', 'TX_RESP_Q225', 'ID_LOCALIZACAO',
#       'PC_FORMACAO_DOCENTE_FINAL', 'NIVEL_SOCIO_ECONOMICO']

# 'TX_RESP_Q118', 'TX_RESP_Q158', 'TX_RESP_Q146',

lista_s = []
lista_p = []
for index in catOrdinais:
    coef_spearmanr, p_valor = spearmanr(mat[index], mat["ESCALA_MT"])
    
    #if round(p_valor, 4) <= 0.05:
    lista_s.append(round(coef_spearmanr, 4))
    lista_p.append(round(p_valor, 4))

ds = pd.DataFrame(list(zip(catOrdinais, lista_s, lista_p)), columns=["Feature", "Spearmanr", "P-Valor"])
ds

,Feature,Spearmanr,P-Valor
0,TX_RESP_Q041,-0.0493,0.0000
1,TX_RESP_Q042,-0.0020,0.8109
2,TX_RESP_Q043,-0.0018,0.8282
3,TX_RESP_Q044,-0.0071,0.3860
4,TX_RESP_Q045,-0.0538,0.0000
5,TX_RESP_Q047,0.0266,0.0012
6,TX_RESP_Q050,-0.0318,0.0001
7,TX_RESP_Q052,0.0179,0.0292
8,TX_RESP_Q150,-0.0025,0.7566
9,TX_RESP_Q151,0.0120,0.1422


### Kendall

In [106]:
#plt.figure(figsize=(12,8))
#sns.heatmap(mat.corr(method='kendall'), annot=True, cmap="PiYG")
#plt.title("Correlação entre Variáveis")
#plt.show()

In [107]:
from scipy.stats.stats import kendalltau

lista_s = []
lista_p = []
for index in catOrdinais:
    coef_kendalltau, p_valor = kendalltau(mat[index], mat["ESCALA_MT"])
    
    #if (p_valor <= 0.05):
    lista_s.append(round(coef_kendalltau, 4))
    lista_p.append(round(p_valor, 4))

ds = pd.DataFrame(list(zip(catOrdinais, lista_s, lista_p)), columns=["Feature", "Kendall", "P-Valor"])
ds

,Feature,Kendall,P-Valor
0,TX_RESP_Q041,-0.0434,0.0000
1,TX_RESP_Q042,-0.0018,0.8037
2,TX_RESP_Q043,-0.0015,0.8391
3,TX_RESP_Q044,-0.0062,0.3841
4,TX_RESP_Q045,-0.0481,0.0000
5,TX_RESP_Q047,0.0232,0.0015
6,TX_RESP_Q050,-0.0288,0.0001
7,TX_RESP_Q052,0.0163,0.0292
8,TX_RESP_Q150,-0.0022,0.7585
9,TX_RESP_Q151,0.0110,0.1418


### Qui-Quadrado: 
testar independendica entre as variáveis
hipotese nula: as variaveis sao de fato independentes

p < 0.05 => rejeita a hipotese nula e concluimos que as variveis sao correlacionadas

p > 0.05 => aceitamos a hipotese nula e concluimos que as variáveis sao independentes

qui-quadrado pode ser aplicado em variaveis categoricas ou númericas

se a variavel for continua é necessario dividir os valores em bins

bins: sao invervalos numericos


In [108]:
from scipy.stats import chi2_contingency

lista_chis = []
lista_p = []
for index in catOrdinais:
    confusion_matrix = pd.crosstab(mat[index], mat["ESCALA_MT"])
    chi2, p_valor, dof, exp = chi2_contingency(confusion_matrix.values)
    if (p_valor <= 0.05):
        lista_chis.append(round(chi2, 4))
        lista_p.append(round(p_valor, 4))
        #print("{} - {} => Qui-Quadrado: {} p-valor: {}".format(index, "ESCALA_MT", chi2, round(p, 4)))

chi2_scores = pd.DataFrame(list(zip(catOrdinais, lista_chis, lista_p)), columns=["Feature", "Chi2", "P-Valor"])
chi2_scores

,Feature,Chi2,P-Valor
0,TX_RESP_Q041,68.0507,0.0
1,TX_RESP_Q042,95.3819,0.0
2,TX_RESP_Q043,177.3426,0.0
3,TX_RESP_Q044,99.9727,0.0
4,TX_RESP_Q045,75.7313,0.0
5,TX_RESP_Q047,304.8545,0.0
6,TX_RESP_Q050,222.4655,0.0
7,TX_RESP_Q052,73.6045,0.0
8,TX_RESP_Q150,82.1254,0.0
9,TX_RESP_Q151,56.7288,0.0
